In [1]:
goal_state = [i for i in range(9)]
goal_state

[0, 1, 2, 3, 4, 5, 6, 7, 8]

In [2]:
import random
initial_state = [i for i in range(9)]
random.shuffle(initial_state)
initial_state

[5, 3, 7, 8, 6, 4, 2, 1, 0]

In [3]:
action_list= dict()
action_list[0] = ['SWR', 'SWB']
action_list[1] = ['SWR', 'SWB', 'SWL']
action_list[2] = ['SWL', 'SWB']
action_list[3] = ['SWU', 'SWB', 'SWR']
action_list[4] = ['SWL', 'SWB', 'SWR', 'SWU']
action_list[5] = ['SWL', 'SWB', 'SWU']
action_list[6] = ['SWR', 'SWU']
action_list[7] = ['SWL', 'SWU', 'SWR']
action_list[8] = ['SWL', 'SWU']

action_list

{0: ['SWR', 'SWB'],
 1: ['SWR', 'SWB', 'SWL'],
 2: ['SWL', 'SWB'],
 3: ['SWU', 'SWB', 'SWR'],
 4: ['SWL', 'SWB', 'SWR', 'SWU'],
 5: ['SWL', 'SWB', 'SWU'],
 6: ['SWR', 'SWU'],
 7: ['SWL', 'SWU', 'SWR'],
 8: ['SWL', 'SWU']}

In [4]:
def SWU(A, ix):
    A[ix], A[ix - 3] = A[ix - 3], A[ix]

def SWB(A, ix):
    A[ix], A[ix + 3] = A[ix + 3], A[ix]

def SWL(A, ix):
    A[ix], A[ix - 1] = A[ix - 1], A[ix]

def SWR(A, ix):
    A[ix], A[ix + 1] = A[ix + 1], A[ix]

In [5]:
def Expand(state):
    ix = state.index(0)
    states = []
    actions = action_list[ix]
    for act in actions:
        copy = [state[i] for i in range(len(state))]
        if act =='SWU':
            SWU(copy, ix)
        elif act == 'SWB':
            SWB(copy, ix)
        elif act == 'SWL':
            SWL(copy, ix)
        elif act == 'SWR':
            SWR(copy, ix)
        else:
            ;
        states.append(copy)
    return states

## BFS

In [6]:
class Queue:
    def __init__(self, initval):
        self.elements = [initval]
    
    def is_empty(self):
        return len(self.elements) == 0
    
    def push(self, val):
        self.elements.append(val)
    
    def pull(self):
        val = self.elements[0]
        self.elements = self.elements[1:]
        return val

In [7]:
def BFS(initial, goal):
    node_state = initial
    if node_state == goal:
        return node_state
    Frontier = Queue(node_state)
    Explored = dict()
    while not Frontier.is_empty():
        node = tuple(Frontier.pull())
        if node == tuple(goal):
            return list(Explored.keys())
        Explored[node] = 1
        successors = [tuple(i) for i in Expand(node)]
        for child in successors:
            if child not in Frontier.elements and child not in Explored.keys():
                Frontier.push(child)
    return False

In [8]:
soln = BFS(initial_state, goal_state)
if soln:
    print("Number of steps is: " + str(len(soln)))
else:
    print(soln)

Number of steps is: 162939


## Greedy Best First Search

In [9]:
class MinHeap(object):
    __slots__ = ['n', 'A']
    #initialise the heap
    def __init__(self, initval):
        self.A = [initval]
        self.n = 0
    
    def printHeap(self):
        ix = 1
        while ix <= self.n:
            print(str(self.A[ix]), end = '')
            ix += 1
            
    def insert(self, newOb):
        self.n += 1
        self.A.append(Element([0, 1, 2, 3, 4, 5, 6, 7, 8], 100))
        ix = self.n
        while(self.A[ix//2] > newOb):
            self.A[ix] = self.A[ix//2]
            ix//=2
        self.A[ix] = newOb
    
    def isEmpty(self):
        if len(self.A) == 0:
            return True
        return False

    def DeleteMin(self):
        if self.n == 0:
            return None
        minval = self.A[1]
        lastval = self.A[self.n]
        self.n = self.n - 1
        i = 1
        while 2*i < self.n :
            child=2*i
            if child != self.n and self.A[child + 1] < self.A[child]:
                child += 1

            if lastval > self.A[child]:
                self.A[i] = self.A[child]
            else:
                break
            i = child
        self.A[i] = lastval
        return minval

    def rewriteNode(self, newOb):
        ix = self.A.index(newOb)
        while self.A[ix//2] > newOb:
            self.A[ix] = self.A[ix//2]
            ix //= 2
        self.A[ix] = newOb

In [10]:
class Element(object):
    __slots__ = ['state', 'cost']
    def __init__(self, state = None, cost = None):
        self.state = state
        self.cost = cost
    
    def __lt__(self, other):
        if self.cost < other.cost:
            return True
        return False
    
    def __gt__(self, other):
        if self.cost > other.cost:
            return True
        return False
    
    def __eq__(self, other):
        if self.state == other.state:
            return True
        return False

In [11]:
def compute_hn(state, goal_state):
    h_n = 0
    for i in range(len(state)):
        if state[i] != goal_state[i]:
            h_n += 1
    return h_n

In [12]:
Frontier = MinHeap(Element([0, 1, 2, 3, 4, 5, 6, 7, 8], -1))

In [13]:
def GreedyBFS(initial, goal):
    node_state = initial
    Frontier.insert(Element(node_state, compute_hn(node_state, goal_state)))
    Explored = dict()
    while not Frontier.isEmpty():
        node = Frontier.DeleteMin()
        Explored[tuple(node.state)] = 1
        if node.cost == 0: 
            return list(Explored.keys())
        successors = [tuple(i) for i in Expand(node.state)]
        for child_state in successors:
            child = Element(child_state, compute_hn(child_state, goal))
            if child not in Frontier.A and child.state not in Explored.keys():
                Frontier.insert(child)
            elif child in Frontier.A:
                Frontier.rewriteNode(child)
    return False

In [14]:
soln = GreedyBFS(initial_state, goal_state)
print("Number of steps is: " + str(len(soln)))

Number of steps is: 780


## A* Search

In [15]:
def compute_fn(state, goal_state, g_n):
    h_n = 0
    for i in range(len(state)):
        if state[i] != goal_state[i]:
            h_n += 1
    f_n = g_n + h_n
    return f_n

In [16]:
Frontier = MinHeap(Element([0, 1, 2, 3, 4, 5, 6, 7, 8], -1))
Explored = dict()

In [17]:
def Astar(initial, goal, level):
    node_state = initial
    node = Element(node_state, compute_fn(node_state, goal_state, level))
    Frontier.insert(node)
    while not Frontier.isEmpty():
        node = Frontier.DeleteMin()
        if node.state == goal: 
            return list(Explored.keys())
        Explored[tuple(node.state)] = 1
        successors = [i for i in Expand(node.state)]
        level += 1
        for child_state in successors:
            child = Element(child_state, compute_fn(child_state, goal_state, level))
            if level > 1000:
                return list(Explored.keys())
            if child not in Frontier.A and tuple(child.state) not in Explored.keys():
                    Frontier.insert(child)
            elif child in Frontier.A:
                Frontier.rewriteNode(child)
    return False

In [18]:
soln = Astar(initial_state, goal_state,0)
print("Number of steps is: " + str(len(soln)))

Number of steps is: 1001
